<a href="https://colab.research.google.com/github/AbhinavSapra/AML_Assignments/blob/main/Assignment_1/prepare_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# We import all the necessary packages
import os
import re
import collections
import matplotlib.pyplot as plt
import csv
from textblob import TextBlob
import pandas as pd
import sklearn
# import Pickle
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import learning_curve

from sklearn.ensemble import RandomForestClassifier

In [31]:
path ="/content/drive/MyDrive/"
os.chdir(path)
data = pd.read_csv(f"{path}/emails.csv")

In [32]:
data.columns


Index(['text', 'spam'], dtype='object')

In [33]:
data['text'].head()

0    Subject: naturally irresistible your corporate...
1    Subject: the stock trading gunslinger  fanny i...
2    Subject: unbelievable new homes made easy  im ...
3    Subject: 4 color printing special  request add...
4    Subject: do not have money , get software cds ...
Name: text, dtype: object

In [34]:
data['spam'].head()

0    1
1    1
2    1
3    1
4    1
Name: spam, dtype: int64

In [35]:
data.describe()

,spam
count,5728.000000
mean,0.238827
std,0.426404
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


Pre-Processing

In [36]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:

def tokenize(text):
  words = word_tokenize(text.lower())
  return words

def pre_process(text):
  words = tokenize(text)
  words = [word for word in words if word.isalpha() and word not in stopwords.words('english')]
  return(' '.join(words))




In [38]:
data['processed_text'] = data['text'].apply(pre_process)
for i in range(3):  # Adjust the range based on the number of examples you want to print
    print("Original:", data['text'][i])
    print("Preprocessed:", data['processed_text'][i])
    print()

Original: Subject: naturally irresistible your corporate identity  lt is really hard to recollect a company : the  market is full of suqgestions and the information isoverwhelminq ; but a good  catchy logo , stylish statlonery and outstanding website  will make the task much easier .  we do not promise that havinq ordered a iogo your  company will automaticaily become a world ieader : it isguite ciear that  without good products , effective business organization and practicable aim it  will be hotat nowadays market ; but we do promise that your marketing efforts  will become much more effective . here is the list of clear  benefits : creativeness : hand - made , original logos , specially done  to reflect your distinctive company image . convenience : logo and stationery  are provided in all formats ; easy - to - use content management system letsyou  change your website content and even its structure . promptness : you  will see logo drafts within three business days . affordability :

In [39]:
from sklearn.model_selection import train_test_split

messages = data['processed_text']
labels = data["spam"]
data['length'] = data['text'].map(lambda text: len(text))


X_train, X_test, y_train, y_test = train_test_split(messages, labels, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [40]:
train_df = pd.DataFrame()
train_df["text"] = X_train
train_df["spam"] = y_train
print("TRAIN DATA")
print(train_df, "\n\n\n")

val_df = pd.DataFrame()
val_df["text"] = X_val
val_df["spam"] = y_val
print("VALIDATION DATA")
print(val_df, "\n\n\n")

test_df = pd.DataFrame()
test_df["text"] = X_test
test_df["spam"] = y_test
print("TEST DATA")
print(test_df)

train_df.to_csv("train.csv", index=False)
val_df.to_csv("validation.csv", index=False)
test_df.to_csv("test.csv", index=False)

TRAIN DATA
                                                   text  spam
2740  subject information shalesh please look give i...     0
1875  subject christmas baskets kevin please add cop...     0
5202  subject credit rating contact kim spoke bill b...     0
3327  subject list teams projects vince list teams p...     0
2207  subject christmas baskets christmas baskets or...     0
...                                                 ...   ...
5651  subject vicky windsor vicky please send resume...     0
1662  subject fw winston debbie update continuing wi...     0
3093  subject alp presentation information alp proje...     0
4123  subject aiesec polska eurolds drogi panie andr...     0
1691  subject phone numbers computer support help co...     0

[3665 rows x 2 columns] 



VALIDATION DATA
                                                   text  spam
3169  subject request submitted access request tony ...     0
2472  subject mscf speaker series kristin gandy foll...     0
28    subject 

In [41]:
train = pd.read_csv("train.csv")
val = pd.read_csv("validation.csv")
test = pd.read_csv("test.csv")

X_train, y_train = train["text"], train["spam"]
X_val, y_val = val["text"], val["spam"]
X_test, y_test = test["text"], test["spam"]

In [42]:
data.head()

,text,spam,processed_text,length
0,Subject: naturally irresistible your corporate...,1,subject naturally irresistible corporate ident...,1484
1,Subject: the stock trading gunslinger fanny i...,1,subject stock trading gunslinger fanny merrill...,598
2,Subject: unbelievable new homes made easy im ...,1,subject unbelievable new homes made easy im wa...,448
3,Subject: 4 color printing special request add...,1,subject color printing special request additio...,500
4,"Subject: do not have money , get software cds ...",1,subject money get software cds software compat...,235
